# Este .ipynb está obsoleto y sólo se dejó por motivos de Archivo y que se vea todo el procedimiento que hice para eventualmente reintentarlo en ETL2 - Intento 2. 

In [30]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus, value

In [31]:
depositos = pd.read_csv('Archivos Intermedios/filtered_depositos.csv')
mayoristas = pd.read_csv('Archivos Intermedios/filtered_mayoristas.csv')
productos = pd.read_csv('Archivos Intermedios/filtered_productos.csv')

In [32]:
depositos

,deposito,capacidad_m3,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5
0,D1_DEP,1500,11.32,11.78,17.97,17.78,15.13
1,D2_DEP,1300,18.22,16.45,16.79,9.80,10.03


In [33]:
mayoristas

,mayorista,D1_DEP,D2_DEP,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5
0,M1,14.88,24.75,29.79,33.88,26.08,27.21,26.10
1,M2,20.04,17.15,32.92,32.76,33.67,36.24,39.47
2,M3,19.22,17.24,30.16,29.16,39.95,36.26,34.67


In [34]:
productos

,producto_id,volumen_m3,costo_M1_usd,costo_M2_usd,costo_M3_usd,pdv_1,pdv_2,pdv_3,pdv_4,pdv_5,demanda_total,volumen_total
0,DEP1,0.232,105.24,95.67,81.32,0,0,0,69,25,94,21.808
1,DEP2,0.180,103.43,94.03,79.93,5,92,7,99,27,230,41.400
2,DEP3,0.186,129.94,118.13,100.41,98,0,0,61,0,159,29.574
3,DEP4,0.156,108.92,99.02,84.17,44,19,0,70,0,133,20.748
4,DEP5,0.211,114.36,103.96,88.37,19,34,26,13,84,176,37.136
...,...,...,...,...,...,...,...,...,...,...,...,...
95,DEP96,0.200,101.31,92.10,78.28,55,30,67,27,0,179,35.800
96,DEP97,0.219,98.49,89.54,76.11,82,45,0,95,32,254,55.626
97,DEP98,0.174,112.12,101.93,86.64,0,72,74,97,29,272,47.328
98,DEP99,0.189,127.64,116.04,98.63,0,36,36,0,17,89,16.821


# ***Texto adicional en inglés para obtener mejores resultados con las IAs cuando tenga dudas***


I have 3 dataframes: mayoristas, productos and depositos

## Mayoristas
Has the following columns:
- mayorista: Represents to which buying center all other attributes correspond. Has only 3 values (or lines); "M1", "M2" and "M3"
- D1_DEP: Transportation cost per cubic meter from the corresponding M to D1
- D2_DEP: Transportation cost per cubic meter from the corresponding M to D2
- pdv_1: Transportation cost per cubic meter from the corresponding M to the selling point 1 (directly, instead of going through deposits)
- pdv_2: Transportation cost per cubic meter from the corresponding M to the selling point 2 (directly, instead of going through deposits)
- pdv_3: Transportation cost per cubic meter from the corresponding M to the selling point 3 (directly, instead of going through deposits)
- pdv_4: Transportation cost per cubic meter from the corresponding M to the selling point 4 (directly, instead of going through deposits)
- pdv_5: Transportation cost per cubic meter from the corresponding M to the selling point 5 (directly, instead of going through deposits)

## Depositos
Has the following columns:
- deposito: Represents to which deposit all other attributes correspond. Has only 2 values (or lines); "D1_DEP", "D2_DEP" and "D3_DEP"
- capacidad_m3: The volume of goods each deposit can hold in cubic meters
- pdv_1 to pdv_5: Cost of transportation per cubic meter from the corresponding deposit to the selected selling point (pdv) shown in USD/(10m^{3})


## Productos
- producto_id: The identifier for each product
- volumen_m3: the volume cubic meters of a unit the corresponding good
- costo_M1_usd: Cost per unit of the corresponding good in USD to the M1 (closer, but more expensive)
- costo_M2_usd: Cost per unit of the corresponding good in USD to the M2 (intermediate)
- costo_M3_usd: Cost per unit of the corresponding good in USD to the M3 (furthest away, but cheapest)
- pdv_1 to 5: Ammount of each product demanded by each selling point (shown in units of the good)
- demanda_total: Sum of the corresponding pdv_1 to 5
- volumen_total: volumen_m3*demanda_total



# What's being asked?
Create 3 csv files representing the buy orders to the 3 buying points (M1, M2 and M3)
The files should have the following columns:
- producto_id: The identifier for each product
- cantidad_total: The quantity that must be bought from the buying point that corresponded to the file. 
- D1_DEP: Quantity of the corresponding good that must be sent to the first deposit
- D1_DEP: Quantity of the corresponding good that must be sent to the second deposit
- pdv_1 to 5: Quantity that must be sent directly from the M represented by this particular file to each selling point (without going through deposits).

In [35]:
# Vuelo a importar todo por si acaso
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, value

In [36]:
# Normalizo los datos de ventas directas sin deposito a USD/m³ en vez de USD/10m³
pdv_columnas = [col for col in depositos.columns if col.startswith("pdv_")]
depositos[pdv_columnas] = depositos[pdv_columnas] / 10

# ***EL PROBLEMA ESTÁ EN ESTOS BLOQUES***

In [ ]:
# Inicializo el problema de puLP (La librería de programación lineal)
problem = LpProblem("Minimize_Cost", LpMinimize)

# Marco mis variables de decisión tal que m es mi mercado, p es mi producto, d es mi deposito y pdv es mi punto de venta
#? Sospecho que el error puede estar acá que haya definido mal las variables de decisión

x = {(m, p, d): LpVariable(f"x_{m}_{p}_{d}", lowBound=0, cat="Continuous")
     for m in ["M1", "M2", "M3"]
     for p in productos["producto_id"]
     for d in depositos["deposito"]}

y = {(m, p, pdv): LpVariable(f"y_{m}_{p}_{pdv}", lowBound=0, cat="Continuous")
     for m in ["M1", "M2", "M3"]
     for p in productos["producto_id"]
     for pdv in [f"pdv_{i}" for i in range(1, 6)]}

In [ ]:
# Armo mi función objetivo/funciona tal que minimizo el costo total de los productos
# ? Si el problema está acá es que estoy armando bien las partes de la función objetivo (poco probable) o tuve un error de no convertir unidades (muy probable)
problem += lpSum(
    productos.loc[productos["producto_id"] == p, f"costo_{m}_usd"].values[0] * (lpSum(x[m, p, d] for d in depositos["deposito"]) + lpSum(y[m, p, pdv] for pdv in [f"pdv_{i}" for i in range(1, 6)]))
    + lpSum(mayoristas.loc[mayoristas["mayorista"] == m, d].values[0] * x[m, p, d] for d in depositos["deposito"])
    + lpSum(mayoristas.loc[mayoristas["mayorista"] == m, pdv].values[0] * y[m, p, pdv] for pdv in [f"pdv_{i}" for i in range(1, 6)])
    for m in ["M1", "M2", "M3"]
    for p in productos["producto_id"]
)

In [ ]:
# Restricciones:
# ? Si El erros fue acá, sospecharía que más que nada puede ser que se calculan mal las cantidades o los volumenes

    # 1. Que cumpla la demanda de cada producto en cada punto de venta
for p in productos["producto_id"]:
    for pdv in [f"pdv_{i}" for i in range(1, 6)]:
        demand = productos.loc[productos["producto_id"] == p, pdv].values[0]
        problem += lpSum(x[m, p, d] for m in ["M1", "M2", "M3"] for d in depositos["deposito"]) + lpSum(y[m, p, pdv] for m in ["M1", "M2", "M3"]) == demand

    # 2. Que no se sobrepase el límite de ningún depósito
for d in depositos["deposito"]:
    capacity = depositos.loc[depositos["deposito"] == d, "capacidad_m3"].values[0]
    problem += lpSum(x[m, p, d] * productos.loc[productos["producto_id"] == p, "volumen_m3"].values[0] for m in ["M1", "M2", "M3"] for p in productos["producto_id"]) <= capacity

# ***Último Bloque Dónde Puede Estar El Problema***

In [1]:
# Resuelvo
problem.solve()

NameError: name 'problem' is not defined

In [ ]:
# Extraigo mis resultados a 3 .csv diferentes, uno por cada mayorista
for m in ["M1", "M2", "M3"]:
    results = []
    for p in productos["producto_id"]:
        row = {
            "producto_id": p,
            "cantidad_total": sum(value(x[m, p, d]) for d in depositos["deposito"]) + sum(value(y[m, p, pdv]) for pdv in [f"pdv_{i}" for i in range(1, 6)]),
        }
        for d in depositos["deposito"]:
            row[d] = value(x[m, p, d])
        for pdv in [f"pdv_{i}" for i in range(1, 6)]:
            row[pdv] = value(y[m, p, pdv])
        results.append(row)
    pd.DataFrame(results).to_csv(f"Resultados/buy_orders_{m}.csv", index=False)

Reviso unos cuantos datos de mis resultados para analizarlos:

In [ ]:
M1 = pd.read_csv("Resultados/buy_orders_M1.csv")
M2 = pd.read_csv("Resultados/buy_orders_M2.csv")
M3 = pd.read_csv("Resultados/buy_orders_M3.csv")

In [ ]:
M1["cantidad_total"].sum()

np.float64(0.0)

In [ ]:
M2["cantidad_total"].sum()

np.float64(0.0)

In [ ]:
M3["cantidad_total"].sum()

np.float64(17380.0)

La totalidad de los productos pasa por M3. Reviso si se satisface la totalidad de la demada

In [ ]:
productos["demanda_total"].sum()

np.int64(18192)

In [ ]:
print(M3['cantidad_total'].sum()-productos["demanda_total"].sum())

-812.0


No estaría llegando a saciar la demanda total con el sistema que armé. faltan 812 unidades en total, distribuídas tal que:

In [ ]:
# Una mini función para ver si hay productos que no se cumplieron
demanda_insatisfecha = []

for p in productos["producto_id"]:
    total_suministrado = sum(
        value(x[m, p, d]) for m in ["M1", "M2", "M3"] for d in depositos["deposito"]
    ) + sum(
        value(y[m, p, pdv]) for m in ["M1", "M2", "M3"] for pdv in [f"pdv_{i}" for i in range(1, 6)]
    )
    demanda_total = productos.loc[productos["producto_id"] == p, "demanda_total"].values[0]
    if total_suministrado < demanda_total:
        demanda_insatisfecha.append({
            "producto_id": p,
            "total_suministrado": total_suministrado,
            "demanda_total": demanda_total,
            "insatisfecha": demanda_total - total_suministrado
        })

# lo hago un df para que se vea más claro
demanda_insatisfecha_df = pd.DataFrame(demanda_insatisfecha)

In [ ]:
demanda_insatisfecha_df

,producto_id,total_suministrado,demanda_total,insatisfecha
0,DEP2,210.0,230,20.0
1,DEP5,124.0,176,52.0
2,DEP6,195.0,283,88.0
3,DEP9,301.0,321,20.0
4,DEP18,227.0,239,12.0
5,DEP25,246.0,286,40.0
6,DEP26,230.0,266,36.0
7,DEP30,197.0,217,20.0
8,DEP57,178.0,186,8.0
9,DEP65,139.0,187,48.0


In [ ]:
demanda_insatisfecha_df["insatisfecha"].sum()

np.float64(812.0)

In [ ]:
# Calcula el costo total de cada mayorista
def calculate_cost(df, market):
    costo_compra = df["producto_id"].apply(
        lambda p: productos.loc[productos["producto_id"] == p, f"costo_{market}_usd"].values[0]
    ) * df["cantidad_total"]
    
    costo_transporte_depositos = sum(
        df[d] * mayoristas.loc[mayoristas["mayorista"] == market, d].values[0]
        for d in depositos["deposito"]
    )
    
    costo_transporte_pdv = sum(
        df[pdv] * mayoristas.loc[mayoristas["mayorista"] == market, pdv].values[0]
        for pdv in [f"pdv_{i}" for i in range(1, 6)]
    )
    
    return costo_compra.sum() + costo_transporte_depositos.sum() + costo_transporte_pdv.sum()

# Calculate total cost for each market
total_cost_M1 = calculate_cost(M1, "M1")
total_cost_M2 = calculate_cost(M2, "M2")
total_cost_M3 = calculate_cost(M3, "M3")

# Sum up all costs
total_cost = total_cost_M1 + total_cost_M2 + total_cost_M3

# Imprimo
print(f"Costo total de M1: {total_cost_M1}")
print(f"Costo total de M2: {total_cost_M2}")
print(f"Costo total de M3: {total_cost_M3}")
print(f"Costo total: {total_cost}")

Costo total de M1: 0.0
Costo total de M2: 0.0
Costo total de M3: 2059587.11
Costo total: 2059587.11


Ahora reviso cuanto va a cada deposito:

In [ ]:
# Calculo la cantidad de productos y sus volumenes por deposito
res_dep = []

for d in depositos["deposito"]:
    cant_total = sum(value(x[m, p, d]) for m in ["M1", "M2", "M3"] for p in productos["producto_id"])
    vol_total = sum(
        value(x[m, p, d]) * productos.loc[productos["producto_id"] == p, "volumen_m3"].values[0]
        for m in ["M1", "M2", "M3"]
        for p in productos["producto_id"]
    )
    res_dep.append({"deposito": d, "cant_total": cant_total, "vol_total": vol_total})

# Lo hago un dataframe para que sea más fácil de leer
resumen_depositos = pd.DataFrame(res_dep)

In [ ]:
resumen_depositos

,deposito,cant_total,vol_total
0,D1_DEP,0.0,0.000
1,D2_DEP,203.0,40.887


Como todos mis productos entran en el dep 2, no hay precio mayor a 0 por el cual estaría dispuesto a aumentar la capacidad de D1.

Corroboro

In [ ]:
Vol_tot = productos["volumen_m3"].sum() * productos["demanda_total"].sum()

In [ ]:
Vol_tot

np.float64(355216.99199999997)

Desapareció la gran mayoría del volúmen. Hay 2 opciones:
- Se satisface una parte minúscula de la demanda (poco probable viendo mi calculo de demanda insatisfecha)
- Hubo un error con las cuentas de volúmen (y potencialmente las de todo lo demás) cuando usé PuLP